# 📘 02 - `@root_validator` in Pydantic

`@root_validator` is used when you want to validate or transform multiple fields **together**.

### 🔍 Why use `@root_validator`?
- Validate interdependent fields
- Enforce cross-field rules
- Perform cleanups using all data

### 🧪 Syntax
```python
@root_validator
def check_all_fields(cls, values):
    ...
    return values
```

### ⚠️ Note: `@root_validator` is Deprecated in Pydantic v2

Use `@model_validator(mode="after")` to validate multiple fields **after** model initialization.

```python
@model_validator(mode="after")
def validate(cls): ...

In [9]:
# Import
from pydantic import BaseModel, root_validator, model_validator
from datetime import date

### 🔐 Validating Conditional Logic with `@model_validator`

When you want to validate relationships between multiple fields (e.g., `is_admin` and `admin_code`), use the `@model_validator` in **Pydantic v2+**.

- `mode="after"` ensures validation happens after the model is built
- You access the **entire object (`self`)**, not just a dictionary of values

#### Example Logic:
- If `is_admin` is `True`, then `admin_code` must be provided
- If not, raise a validation error

This pattern is clean and scalable for enforcing conditional rules across fields.

## 🗓️ Validate that end_date is after start_date

In [10]:
class Event(BaseModel):
    name: str
    start_date: date
    end_date: date

    @model_validator(mode="after")
    def check_dates(self):
        if self.end_date < self.start_date:
            raise ValueError("end_date must be after start_date")
        return self

# ✅ Valid event
e = Event(name="Workshop", start_date="2025-07-01", end_date="2025-07-10")
print(e)

# ❌ Invalid event: end_date before start_date
try:
    bad = Event(name="Hackathon", start_date="2025-08-01", end_date="2025-07-10")
except Exception as err:
    print(err)

name='Workshop' start_date=datetime.date(2025, 7, 1) end_date=datetime.date(2025, 7, 10)
1 validation error for Event
  Value error, end_date must be after start_date [type=value_error, input_value={'name': 'Hackathon', 'st...end_date': '2025-07-10'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


## 🔐 Example: If `is_admin=True`, `admin_code` must be provided

In [11]:
class User(BaseModel):
    username: str
    is_admin: bool = False
    admin_code: str | None = None

    @model_validator(mode="after")
    def check_admin_code(self):
        if self.is_admin and not self.admin_code:
            raise ValueError("Admin code is required for admin users")
        return self

# ✅ Valid
u = User(username="gigi", is_admin=True, admin_code="ADM123")
print(u)

# ❌ Invalid: is_admin but no admin_code
try:
    bad = User(username="gigi", is_admin=True)
except Exception as err:
    print(err)

username='gigi' is_admin=True admin_code='ADM123'
1 validation error for User
  Value error, Admin code is required for admin users [type=value_error, input_value={'username': 'gigi', 'is_admin': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


## ✅ Summary: `@model_validator` (Pydantic v2)

| Feature                   | Use Case                                    |
|---------------------------|---------------------------------------------|
| `@model_validator`        | Validate across multiple fields             |
| `self` access             | Receives the entire model instance          |
| `mode="after"`            | Runs after all fields are validated         |
| Must `return self`        | To retain the validated model instance      |
| `raise ValueError`        | To raise a custom validation error          |

Use `@model_validator` when the relationship **between multiple fields** matters.

> ⚠️ Note: `@root_validator` is **deprecated** in Pydantic v2.